In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler
import keras
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Activation
from keras.optimizers import RMSprop
from keras.utils import np_utils
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
import numpy as np
from keras.layers import Dense, Dropout
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import classification_report, precision_score

datasets = pd.read_csv('fish.csv',encoding='unicode_escape')
datasets.columns=['Sex',
              'Length',
              'Diameter',
              'Height',
              'Whole weight',
              'Shucked weight',
              'Viscera weight',
              'Shell weight',
              'Rings'
              ]

encoder = LabelEncoder() 
encoded_y = encoder.fit_transform(datasets['Sex'])
dummy_y = keras.utils.to_categorical(encoded_y, 3) # 001 100 010

datasets.drop_duplicates()
(datasets.astype(str) == ' ?').any(1)
datasets = datasets[~(datasets.astype(str) == ' ?').any(1)]
lack = datasets.isnull()

x=datasets[["Length","Diameter","Height","Whole weight","Rings"]]
y=datasets[['Sex']]

x_train,x_test,y_train,y_test = train_test_split(x,dummy_y,test_size=0.2,random_state=90)
scaler = MinMaxScaler()
scaler.fit(x_train) 
 
scaled_xtrain_data = scaler.transform(x_train) 
scaler_xtest_data = scaler.transform(x_test) 

scaler = MinMaxScaler() 
scaler.fit(y_train) 
 
scaled_ytrain_data = scaler.transform(y_train) 
scaler_ytest_data = scaler.transform(y_test) 

model = Sequential()
model.add(Dense( output_dim = 100, input_dim = 5, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(50, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(output_dim = 3, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy',  optimizer = 'adam',  metrics=['accuracy'])
model.fit(scaled_xtrain_data,scaled_ytrain_data , batch_size=32, epochs=10)
score = model.evaluate(scaler_xtest_data, scaler_ytest_data)
print(score)

pred = model.predict(scaler_xtest_data)
pred = (pred > 0.5)
print(classification_report(scaler_ytest_data, pred, digits=4)) # 836,3